In [1]:
import os

In [2]:
%pwd

'/Users/chaitanya/Documents/ML/github/Kideny-Disease-Classification-Deep-Learning-Project-DS/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/chaitanya/Documents/ML/github/Kideny-Disease-Classification-Deep-Learning-Project-DS'

In [ ]:
import dagshub
dagshub.init(repo_owner='susmiti63', repo_name='Kideny-Disease-Classification-Deep-Learning-Project-DS', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

In [9]:
import dagshub
dagshub.init(repo_owner='susmiti63', repo_name='Kideny-Disease-Classification-Deep-Learning-Project-DS', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

Initialized MLflow to track repo "susmiti63/Kideny-Disease-Classification-Deep-Learning-Project-DS"

Repository susmiti63/Kideny-Disease-Classification-Deep-Learning-Project-DS initialized!

In [12]:
import dagshub
dagshub.init(repo_owner='susmiti63',
             repo_name='Kideny-Disease-Classification-Deep-Learning-Project-DS',
             mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

Initialized MLflow to track repo "susmiti63/Kideny-Disease-Classification-Deep-Learning-Project-DS"

Repository susmiti63/Kideny-Disease-Classification-Deep-Learning-Project-DS initialized!

In [50]:
import tensorflow as tf

In [51]:
model = tf.keras.models.load_model("artifacts/training/model.h5")


In [52]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [53]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [54]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/susmiti63/Kideny-Disease-Classification-Deep-Learning-Project-DS.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [70]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [71]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [72]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-08-18 01:57:49,706: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-08-18 01:57:49,710: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-18 01:57:49,712: INFO: common: created directory at: artifacts]
Found 139 images belonging to 2 classes.
9/9 [==============================] - 14s 2s/step - loss: 0.0514 - accuracy: 0.9712
[2024-08-18 01:58:04,034: INFO: common: json file saved at: scores.json]


2024/08/18 01:58:04 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /var/folders/vy/lbm6tmp563jbdts82tm6d2ym0000gn/T/tmpvvako7jr/model/data/model/assets
[2024-08-18 01:58:05,204: INFO: builder_impl: Assets written to: /var/folders/vy/lbm6tmp563jbdts82tm6d2ym0000gn/T/tmpvvako7jr/model/data/model/assets]


AttributeError: module 'tensorflow.keras' has no attribute '__version__'

In [73]:
import tensorflow as tf
from tensorflow import keras
tf.keras.__version__

AttributeError: module 'tensorflow.keras' has no attribute '__version__'

In [36]:
!pip uninstall keras
y


Found existing installation: keras 2.13.1
Uninstalling keras-2.13.1:
  Would remove:
    /opt/anaconda3/envs/kidney/lib/python3.8/site-packages/keras-2.13.1.dist-info/*
    /opt/anaconda3/envs/kidney/lib/python3.8/site-packages/keras/*
Proceed (Y/n)? ^C
ERROR: Operation cancelled by user
